# HPE Container Platform API series  - Lab 2
## Deploying cloud-native and non-cloud-native applications, programmatically on Kubernetes clusters managed by the HPE Container Platform.


**Requirements:**
- HPE Container Platform deployment with a managed Kubernetes cluster
- IP address or FQDN of the HPE Container Platform's controller host and Gateway host
- a Kubernetes tenant user account   

**Utilities:**   
- cURL  
- Jupyter Notebook server with bash kernel installed
- kubectl


**Lab workflow:**

In this lab part:

1. As tenant user you will first establish a valid login session with HPE CP using the HPE CP REST API.

2. You will then fetch the Kubeconfig file for your tenant working context using the HPE CP REST API.
>Note: A kubeconfig file is used to access the Kubernetes cluster for your tenant working context

3. Finally, using a K8s API client such as **kubectl** you will deploy a simple cloud-native *stateless*, microservices applications, and a non-cloud-native distributed *stateful* AI/analytics **KubeDirector** application on **Kubernetes** cluster managed by the HPE Container Platform. 
>Note: kubectl has been installed in the JupyterHub server by the IT administrator.

**Definitions:**

- *KubeDirector:* also known as Kubernetes Director, is a key component of HPE Container Platform. KubeDirector is an open source project initiated by HPE (BlueData) that enables the running of non-cloud-native stateful monolithic analytics workloads on Kubernetes. In HPE Container Platform, these applications generally refer to a distributed, single-node or multi-node application **virtual cluster**. Each application virtual cluster node runs as **a container** in the HPE Container Platform.

- *Cloud-native application:* Also known as the 12 Factor app, a modern application that leverages microservices architecture with loosely coupled services.

- *Non-cloud-native application:* a multi-tier application with tightly coupled and interdependent services. 

- *Stateless application:* Stateless application is an application which does not require persistence of data and application state.

- *Stateful application:* A stateful application typically requires persistence of certain mountpoints across application cluster nodes rescheduling, restarts, upgrades, rollbacks. A stateful application can also need persistence of network identity (i.e.: hostname). 

#### Initialize the environment.

Let's first define the environment variables according to your HPE Container Platform working context (credentials and tenant name), and the HPE Container Platform API system endpoint:

In [1]:
#
# environment variables to be adjusted/verified by the student
#
student="student1" # your Jupyter Notebook student Identifier (i.e.: student<xx>)
username="student1" # your HPE CP tenant login credentials - username 
password="stuASP2020" # your HPE CP tenant login credentials - password
#
# fixed environment variables setup by the HPE CP lab administrator
#
controller_endpoint="gateway1.hpedevlab.net:8080"
gateway_host="gateway1.hpedevlab.net"
tenantname="K8sHackTenant"
k8sClusterId="1"  #this is the K8s Cluster Id provided by the HPE CP admisnistrator and assigned to your K8s tenant.
helloWorldApp="hello-world-app.yaml" # the application manifest you will deploy in this lab
tensorFlowApp="tensorflow-notebook-config-cluster.yaml" # the kubedirector application cluster configuration TensorFlow

echo "your working context is:" $username-$password-$tenantname 

your working context is: student1-stuASP2020-K8sHackTenant


#### Authenticate as Tenant user in the specified tenant:

In [2]:
sessionlocation=$(curl -k -i -s --request POST "https://${controller_endpoint}/api/v2/session" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
"name": "'"$username"'",
"password": "'"$password"'",
"tenant_name": "'"$tenantname"'"
}' | grep Location | awk '{print $2}' | tr -d '\r') #we remove any cr that might exist
echo "This is your session location: " $sessionlocation
SessionId=$(echo $sessionlocation | cut -d'/' -f 5) # extract sessionId for later, for logout
echo "This is your session_Id:" $SessionId

This is your session location:  /api/v2/session/6d272579-7d6e-4742-bc18-523ba717d64f
This is your session_Id: 6d272579-7d6e-4742-bc18-523ba717d64f


#### Make a quick check to ensure you can make REST API calls within your tenant working context:
Here you will fetch information about your session you have just established.

In [3]:
curl -k -s --request GET "https://${controller_endpoint}/api/v2/session/${SessionId}" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' | jq  #using jq to pretty print the JSON reponse of the API call 

{
  "_links": {
    "self": {
      "href": "/api/v2/session/6d272579-7d6e-4742-bc18-523ba717d64f"
    },
    "all_sessions": {
      "href": "/api/v2/session"
    }
  },
  "user": "/api/v1/user/1155",
  "user_name": "student1",
  "tenant": "/api/v1/tenant/5",
  "tenant_name": "K8sHackTenant",
  "role": "/api/v1/role/3",
  "role_name": "Member",
  "expiry": "2020-5-18 19:24:15",
  "expiry_time": 1589822655,
  "is_site_admin_view": false,
  "is_cluster_superuser": false
}


#### Get the Kubeconfig file for your tenant working context
The next step in deploying a containerized application in Kubernetes clusters managed by the HPE Container Platform, is to get the kubeconfig file for your tenant working context. 

The HPE CP REST API call below allows you to get the **kubeconfig file** used to access the Kubernetes cluster for your tenant user account based on your assigned role (tenant member), with the same result as if you had downloaded it from the HPE CP UI.

In [4]:
curl -k -s --request GET "https://${controller_endpoint}/api/v2/k8skubeconfig" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' > ${student}_kubeconfig

In [7]:
cat ${student}_kubeconfig

apiVersion: v1
clusters:
- cluster:
    insecure-skip-tls-verify: true
    server: https://gateway1.etc.fr.comm.hpecorp.net:9500
  name: k8scluster1
contexts:
- context:
    cluster: k8scluster1
    user: HPECP-student1
    namespace: k8shacktenant
  name: k8scluster1-K8sHackTenant-student1
current-context: k8scluster1-K8sHackTenant-student1
kind: Config
preferences: {}
users:
- name: HPECP-student1
  user:
    exec:
      command: kubectl
      apiVersion: client.authentication.k8s.io/v1beta1
      args:
      - hpecp
      - authenticate
      - gateway1.etc.fr.comm.hpecorp.net:8080
      - --hpecp-user=student1
      - --hpecp-token=/api/v2/session/6d272579-7d6e-4742-bc18-523ba717d64f
      - --hpecp-token-expiry=1589822655
      - --force-reauth=false
      - --insecure-skip-tls-verify=true


> Notice the kubeconfig file includes the valid token (session location) for your current session you previously established.

##### Define the Kubeconfig file as a shell environment variable
The kubectl command-line tool (a K8s API client) uses kubeconfig file to communicate with the Kube API server of a Kubernetes cluster. By default, kubectl looks for a file named **config** in the $HOME/.kube directory. Because our kubeconfig file is not located in default location, we must specify here the path of the kubeconfig file by setting the KUBECONFIG environment variable:

In [8]:
#define the Kubeconfig file as a shell environment variable to tell kubectl where to look for the kubeconfig file
export KUBECONFIG="${student}_kubeconfig"
echo $KUBECONFIG

student1_kubeconfig


##### Check your working tenant context

In [9]:
kubectl config view

apiVersion: v1
clusters:
- cluster:
    insecure-skip-tls-verify: true
    server: https://gateway1.etc.fr.comm.hpecorp.net:9500
  name: k8scluster1
contexts:
- context:
    cluster: k8scluster1
    namespace: k8shacktenant
    user: HPECP-student1
  name: k8scluster1-K8sHackTenant-student1
current-context: k8scluster1-K8sHackTenant-student1
kind: Config
preferences: {}
users:
- name: HPECP-student1
  user:
    exec:
      apiVersion: client.authentication.k8s.io/v1beta1
      args:
      - hpecp
      - authenticate
      - gateway1.etc.fr.comm.hpecorp.net:8080
      - --hpecp-user=student1
      - --hpecp-token=/api/v2/session/6d272579-7d6e-4742-bc18-523ba717d64f
      - --hpecp-token-expiry=1589822655
      - --force-reauth=false
      - --insecure-skip-tls-verify=true
      command: kubectl
      env: null


You can now send Kubernetes API requests using a K8s API client such as **kubectl** to deploy enterprise workloads to the kubernetes cluster using the privileges assigned to your tenant role. 

>Note: kubectl has been installed in the JupyterHub server by the IT administrator.

**Let's see this in action!**

## Deploying a simple cloud-native stateless application

#### -1- Deploy Hello World containerized application
You will deploy a simple Hello World application on the Kubernetes cluster made available to your tenant. The K8s API call requires the kubectl operation type (create or apply) and the application manifest (YAML file that describes the attributes of the application).  

The hello-world application is a **stateless** application because it does not require persistence of data and application state. The hello-world application is a very simple application that will return `Hello Kubernetes!` when accessed. 

As you are all sharing the same tenant context and Kubernetes cluster resources, let's make sure your application deployment name will be unique among the tenant users. Here we replace the string "example" with your "username".

In [10]:
sed -i "s/example/${username}/g" $helloWorldApp
cat $helloWorldApp

apiVersion: apps/v1
kind: Deployment
metadata:
  name: hello-world-student1
spec:
  selector:
    matchLabels:
      run: hello-student1
  replicas: 2
  template:
    metadata:
      labels:
        run: hello-student1
    spec:
      containers:
        - name: hello-world-student1
          image: gcr.io/google-samples/node-hello:1.0
          ports:
            - containerPort: 8080
              protocol: TCP
---
apiVersion: v1
kind: Service
metadata:
  name: hello-world-service-student1
spec:
  selector:
    run: hello-student1
  ports:
  - name: http-hello
    protocol: TCP
    port: 8080
    targetPort: 8080
  type: NodePort


The YAML file describes the application resources to deploy such as the Kubernetes Deployment, the Pod, the container Docker image and port, and the NodePort service used to expose the application outside the Kubernetes cluster.

In [11]:
kubectl apply -f $helloWorldApp

deployment.apps/hello-world-student1 created
service/hello-world-service-student1 created


After a minute or so, you should get the response message to your K8s API request: *Deployment and service created* on the Kubernetes cluster.   

In the next steps, you will use kubectl commands in the context of your tenant user account and get the service endpoints of your application to connect to it.

#### -2- Get the Kubernetes Pod and Service for your deployed application:

In [12]:
kubectl get deploy,pod,service | grep ${username}

deployment.apps/hello-world-student1   2/2     2            2           11s
pod/hello-world-student1-6bb95447f8-qvc95   1/1     Running   0          11s
pod/hello-world-student1-6bb95447f8-zggck   1/1     Running   0          11s
service/hello-world-service-student1   NodePort   10.96.194.242   <none>        8080:31949/TCP   11s


In [14]:
kubectl describe service hello-world-service-${username}

Name:                     hello-world-service-student1
Namespace:                k8shacktenant
Labels:                   hpecp.hpe.com/hpecp-internal-gateway=true
Annotations:              hpecp-internal-gateway/8080: gateway1.etc.fr.comm.hpecorp.net:10002
                          kubectl.kubernetes.io/last-applied-configuration:
                            {"apiVersion":"v1","kind":"Service","metadata":{"annotations":{},"name":"hello-world-service-student1","namespace":"k8shacktenant"},"spec"...
Selector:                 run=hello-student1
Type:                     NodePort
IP:                       10.96.194.242
Port:                     http-hello  8080/TCP
TargetPort:               8080/TCP
NodePort:                 http-hello  31949/TCP
Endpoints:                10.192.0.141:8080,10.192.1.28:8080
Session Affinity:         None
External Traffic Policy:  Cluster
Events:
  Type    Reason  Age   From         Message
  ----    ------  ----  ----         -------
  Normal  HpeCp   32s  

#### -3- Get the gateway mapped service endpoint
HPE Container Platform automatically maps the **NodePort Service** endpoint to the HPE Container Platform gateway (proxy) host.
Access to application services running in containers is proxied via the gateway host on a publicly-accessible IP address and a port number greater than 10000.

In [15]:
myservice="hello-world-service-"${username}
appURL=$(kubectl describe service/"${myservice}" | grep gateway1 | awk '{print $3}')
appPort=$(echo $appURL | cut -d':' -f 2) # extract the gateway re-mapped port value.
myapp_endpoint="https://$gateway_host:$appPort"
echo "Your application service endpoint re-mapped port is: "$appPort
echo "Your Intranet application service endpoint is: "$myapp_endpoint

Your application service endpoint re-mapped port is: 10002
Your Intranet application service endpoint is: https://gateway1.hpedevlab.net:10002


#### -4- Check your application is responding:

In [16]:
curl -k -s "${myapp_endpoint}"

Hello Kubernetes!

>Note: You can also connect to your application from your browser through our Internet NAT firewall:   
  Open a new tab in your browser, and connect to the service endpoint: https://notebooks.hpedev.io:YourPortNumber where port number is the re-mapped port you get above for your service endpoint.   
Do a copy (CTRL-C) of the URL after execution of the cell code below, and paste it (CTRL-V) on a new tab in your browser.

In [17]:
echo "Your application service is also accessible from your browser at:" "https://notebooks.hpedev.io:$appPort"

Your application service is also accessible from your browser at: https://notebooks.hpedev.io:10002


#### -5- Delete your stateless application deployment
Delete your application deployment and services using the K8s API request below. The K8s API call requires the kubectl operation type (delete) and the application YAML manifest.
After a minute or so, you should get the message: deployment deleted and service deleted.

In [18]:
kubectl delete -f $helloWorldApp

deployment.apps "hello-world-student1" deleted
service "hello-world-service-student1" deleted


## Deploying a KubeDirector Application

HPE has been working within the open source Kubernetes community to add capabilities that enable the running of **stateful** analytics workloads (e.g: data-intensive, AI/ML and analytics-processing distributed applications) on Kubernetes.  The open source project is known as **Kubernetes Director** or **KubeDirector** for short.   
With KubeDirector, enterprises can deploy all their enterprise applications on a common Kubernetes framework.
KubeDirector is deployed as a custom controller (aka an operator) by default on every Kubernetes cluster managed by the HPE Container Platform.

For more information about KubeDirector, check out the HPE DEV portal [here](https://developer.hpe.com/platform/hpe-container-platform/home).

#### -1- List the installed KubeDirector Application
In the current release of HPE Container Platform, the Kubernetes clusters managed by HPE Controller platform has three (3) pre-configured KubeDirector Application types installed out of the box. A tenant user who would access the HPE Container Platform portal UI could see it:

<img src="Pictures/HPECP-KubeDirectorApp-GUI.png" height="800" width="600" align="left">

You can also get the list of KubeDirector Applications using kubectl command:

In [19]:
kubectl get kubedirectorapp

NAME                  AGE
centos7x              54d
ml-jupyter-notebook   54d
spark221e2            54d


A KubeDirector Application describes an application metadata: the service roles, the service endpoints port and port name prefix (that comes from the URL Scheme), the Docker images, the app setup packages, the cardinality (minimum number of members in a role), and if appropriate, the root file system directories (e.g.: /etc, /bin, /opt, /var, /usr) of the containers to persist beyond the life span of the containers. This means stateful applications that require to write data to their root file systems can now successfully run on Kubernetes. 

Let's inspect a couple of KubeDirector Application type definitions: 

In [21]:
kubectl describe kubedirectorapp ml-jupyter-notebook 

Name:         ml-jupyter-notebook
Namespace:    k8shacktenant
Labels:       <none>
Annotations:  token:
                execute this command to get the authentication token 'kubectl exec <pod-name> jupyter notebook list' and use this token in Jupyter noteboo...
API Version:  kubedirector.hpe.com/v1beta1
Kind:         KubeDirectorApp
Metadata:
  Creation Timestamp:  2020-03-24T08:25:15Z
  Generation:          2
  Resource Version:    2874116
  Self Link:           /apis/kubedirector.hpe.com/v1beta1/namespaces/k8shacktenant/kubedirectorapps/ml-jupyter-notebook
  UID:                 cb7a6132-f321-45d6-81f5-635f0b79e61e
Spec:
  Config:
    Role Services:
      Role ID:  controller
      Service I Ds:
        jupyter-nb
    Selected Roles:
      controller
  Config Schema Version:   7
  Default Config Package:  <nil>
  Distro ID:               bluedata/tensorflow
  Label:
    Description:  TensorFlow GPU with Jupyter notebook
    Name:         TensorFlow + Jupyter
  Roles:
    Cardinality:

In [22]:
kubectl describe kubedirectorapp spark221e2

Name:         spark221e2
Namespace:    k8shacktenant
Labels:       <none>
Annotations:  <none>
API Version:  kubedirector.hpe.com/v1beta1
Kind:         KubeDirectorApp
Metadata:
  Creation Timestamp:  2020-03-24T08:25:15Z
  Generation:          1
  Resource Version:    981021
  Self Link:           /apis/kubedirector.hpe.com/v1beta1/namespaces/k8shacktenant/kubedirectorapps/spark221e2
  UID:                 753618be-1edf-470c-bff5-385d0e76fafe
Spec:
  Config:
    Role Services:
      Role ID:  controller
      Service I Ds:
        ssh
        spark
        spark-master
        spark-worker
      Role ID:  worker
      Service I Ds:
        ssh
        spark-worker
      Role ID:  jupyter
      Service I Ds:
        ssh
        jupyter-nb
    Selected Roles:
      controller
      worker
      jupyter
  Config Schema Version:  7
  Distro ID:              bluedata/spark221e2
  Label:
    Description:  Spark 2.2.1 with Jupyter notebook
    Name:         Spark 2.2.1 + Jupyter
  Roles:
   

#### -2- Deploy the TensorFlow with Jupyter Notebook stateful application
A configuration manifest YAML file is used to create KubeDirector virtual clusters that **instantiate** the defined KubeDirector App type. The configuration file is used to describe the attributes of a given KubeDirector App.

We need to make sure the instance name of the KubeDirector Application is unique among your tenant. 

In [23]:
sed -i "s/example/${username}/g" $tensorFlowApp
cat $tensorFlowApp

---
apiVersion: "kubedirector.hpe.com/v1beta1"
kind: "KubeDirectorCluster"
metadata: 
  name: "ml-jupyter-notebook-student1"
spec: 
  app: "ml-jupyter-notebook"
  appCatalog: "local"
  roles: 
    - 
      id: "controller"
      members: 1
      resources: 
        requests: 
          memory: "4Gi"
          cpu: "2"
          nvidia.com/gpu: "0"
        limits: 
          memory: "4Gi"
          cpu: "2"
          nvidia.com/gpu: "0"


The configuration manifest specifies the application instance name, the KubeDirectorApp type, the application service roles to deploy, the number of members (nodes) in a role and compute size, as well as the storage size if the application requires persistent storage. 

>Note: The AI/ML TensorFlow application is the easiest example to understand. The TensorFlow KubeDirector Application variant used in this tutorial specifies a single node member and it does not specify any persistent storage. Jupyter Notebook is embedded in this local implementation and it is used as interactive client to execute AI/ML programs.

Let's deploy it now using the K8s API call below. The API call requires the **kubectl** operation type (create or apply) and the application YAML manifest.

In [24]:
kubectl apply -f $tensorFlowApp

kubedirectorcluster.kubedirector.hpe.com/ml-jupyter-notebook-student1 created


After a minute or so, you should get the response message to your K8s API request: *kubedirectorcluster/Your-instance-name created*.  

Once you get this message, go to the next step.

#### -3- Inspect the deployed KubeDirector application virtual cluster 
Your application virtual cluster will be represented in the Kubernetes cluster by a resource of type **KubeDirectorCluster**, with the name that was indicated inside the YAML file used to create it. **A kubeDirectorCluster resource is an instantiation of a KubeDirector App**.

In [25]:
clusterName="ml-jupyter-notebook-${username}"
kubectl get kubedirectorcluster $clusterName

NAME                           AGE
ml-jupyter-notebook-student1   18s


After creating the instance of the KubeDirector Application, you can use kubectl command below to observe its status and the standard Kubernetes resources that compose the application virtual cluster (statefulsets, pods, services, persistent volume claim if any) and any events logged against it.

The virtual cluster status indicates its overall "state" (top-level property of the status object). It should have a value of **"configured"**. 

> Note: The first time a virtual cluster of a given KubeDirector App type is created, it may take some minutes to reach "configured" state, as the relevant Docker image must be downloaded and imported.

In [26]:
kubectl describe kubedirectorcluster $clusterName

Name:         ml-jupyter-notebook-student1
Namespace:    k8shacktenant
Labels:       <none>
Annotations:  kubectl.kubernetes.io/last-applied-configuration:
                {"apiVersion":"kubedirector.hpe.com/v1beta1","kind":"KubeDirectorCluster","metadata":{"annotations":{},"name":"ml-jupyter-notebook-student...
API Version:  kubedirector.hpe.com/v1beta1
Kind:         KubeDirectorCluster
Metadata:
  Creation Timestamp:  2020-05-17T17:37:46Z
  Finalizers:
    kubedirector.hpe.com/cleanup
  Generation:        1
  Resource Version:  12480716
  Self Link:         /apis/kubedirector.hpe.com/v1beta1/namespaces/k8shacktenant/kubedirectorclusters/ml-jupyter-notebook-student1
  UID:               399a167f-9b01-4fc5-8327-41612f8d1f93
Spec:
  App:          ml-jupyter-notebook
  App Catalog:  local
  Roles:
    Id:       controller
    Members:  1
    Resources:
      Limits:
        Cpu:             2
        Memory:          4Gi
        nvidia.com/gpu:  0
      Requests:
        Cpu:            

In [27]:
kubectl get all -l kubedirector.hpe.com/kdcluster=$clusterName

NAME               READY   STATUS    RESTARTS   AGE
pod/kdss-z96rj-0   1/1     Running   0          32s

NAME                     TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)          AGE
service/kdhs-vm4wq       ClusterIP   None           <none>        8888/TCP         32s
service/s-kdss-z96rj-0   NodePort    10.96.41.195   <none>        8888:32489/TCP   32s

NAME                          READY   AGE
statefulset.apps/kdss-z96rj   1/1     32s


Your TensorFlow KubeDirector application virtual cluster is made up of a **StatefulSet**, a **POD** (a cluster node) per service role member, a **NodePort Service** per service role member and a **headless service** for the application cluster.   

* The ClusterIP service is the headless service required by a Kubernetes StatefulSet to work. It maintains a stable POD network identity (i.e.: persistence of the hostname of the PODs across PODs rescheduling).
* The NodePort service exposes an application service outside the Kubernetes cluster. 
HPE Container Platform automatically maps the NodePort Service endpoint to the HPE Container Platform gateway (proxy) host.

#### -4- Get the gateway mapped application service endpoint
To get a report on all services related to a specific virtual cluster, you can use a form of "kubectl describe" that matches against a value of the **kubedirector.hpe.com/kdcluster=YourClusterApplicationName** label.

In [28]:
tfappURL=$(kubectl describe service -l  kubedirector.hpe.com/kdcluster=${clusterName} | grep gateway1 | awk '{print $3}')
tfappPort=$(echo $tfappURL | cut -d':' -f 2) # extract the gateway re-mapped port value.
mytfapp_endpoint="https://$gateway_host:$tfappPort"
echo "Your application service endpoint re-mapped port is: "$tfappPort
echo "Your Intranet application service endpoint is: "$mytfapp_endpoint

Your application service endpoint re-mapped port is: 10002
Your Intranet application service endpoint is: https://gateway1.hpedevlab.net:10002


Access to application services running in containers is proxied via the gateway host and a port number greater than 10000.

#### -5- Verify the application service is responding

In [29]:
curl -k -L -s -i "${mytfapp_endpoint}" | grep "HTTP/1.1"

HTTP/1.1 302 Found
HTTP/1.1 302 Found
HTTP/1.1 200 OK


#### -6- Fetch the authentication token of TensorFlow Jupyter Notebook 

Execute this command to get the authentication token: `kubectl exec pod-name jupyter notebook list` and use this token to login to your TensorFlow Jupyter notebook:

In [30]:
mytfapp_pod=$(kubectl get pod -l kubedirector.hpe.com/kdcluster=${clusterName} | grep kdss | awk '{print $1}') 
mytfapp_token=$(kubectl exec ${mytfapp_pod} jupyter notebook list)
mytfToken=$(echo $mytfapp_token | cut -d'?' -f 2 | cut -d':' -f 1) # extract the Token value
echo "Your TensorFlow Jupyter Notebook login token is:" $mytfToken

Your TensorFlow Jupyter Notebook login token is: token=9bef50bf7b90deae4154153d8aa7d9acce7ac2ebe51c827b


#### -7- Connect to your application from your browser

You can connect to your tensorflow framework with Jupyter Notebook and start build your ML model (for example a classification model), train the model and make prediction.

Open a new tab in your browser, and connect to the application service endpoint through our Internet NAT firewall: https://notebooks.hpedev.io:YourPortNumber where port number is the re-mapped port you get for your service endpoint. When prompted to enter your password or token, please enter the authentication token you have just fetched in the previous step.

>Note: You can also do a copy (CTRL-C) of the URL after execution of the cell code below, and paste it (CTRL-V) on a new tab in your browser.

In [31]:
echo "Your application service URL:" "https://notebooks.hpedev.io:$tfappPort?$mytfToken"

Your application service URL: https://notebooks.hpedev.io:10002?token=9bef50bf7b90deae4154153d8aa7d9acce7ac2ebe51c827b 


![TensorFlow Jupyter Notebook](Pictures/kubedirector-tf-jupyterNotebook.png)

#### -8- Go through some cleanup

* First delete the kubedirector application cluster instance using the K8s API call below. The K8s API call requires the **kubectl** operation type (delete) and the application manifest.

> Note: Deleting the KubeDirectorCluster resource will automatically delete all resources (for example the pods, services, statefulset) that compose the KubeDirector application virtual cluster.

In [32]:
kubectl delete -f $tensorFlowApp

kubedirectorcluster.kubedirector.hpe.com "ml-jupyter-notebook-student1" deleted


* Although session have a time to live (TTL) of 24 hours, it is best practice in REST API programming to cleanup and delete those sessions when done. We can use a DELETE /api/v2/session/SessionId to achieve this.

In [33]:
curl -k -i -s --request DELETE "https://${controller_endpoint}/api/v2/session/${SessionId}" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json'

HTTP/1.1 204 No Content
Access-Control-Allow-Origin: *
Content-Length: 0
Content-Type: text/plain
Date: Sun, 17 May 2020 17:40:18 GMT
Server: BlueData EPIC 5.0



The status *204 No Content* means the session object has been deleted.

* Finally, reset your applications YAML files and delete the kubeconfig file

In [34]:
#reset the application deployment name in the YAML file and delete the kubeconfgi file
rm ${student}_kubeconfig
sed -i "s/${username}/example/g" $helloWorldApp
sed -i "s/${username}/example/g" $tensorFlowApp
cat $helloWorldApp
cat $tensorFlowApp

apiVersion: apps/v1
kind: Deployment
metadata:
  name: hello-world-example
spec:
  selector:
    matchLabels:
      run: hello-example
  replicas: 2
  template:
    metadata:
      labels:
        run: hello-example
    spec:
      containers:
        - name: hello-world-example
          image: gcr.io/google-samples/node-hello:1.0
          ports:
            - containerPort: 8080
              protocol: TCP
---
apiVersion: v1
kind: Service
metadata:
  name: hello-world-service-example
spec:
  selector:
    run: hello-example
  ports:
  - name: http-hello
    protocol: TCP
    port: 8080
    targetPort: 8080
  type: NodePort
---
apiVersion: "kubedirector.hpe.com/v1beta1"
kind: "KubeDirectorCluster"
metadata: 
  name: "ml-jupyter-notebook-example"
spec: 
  app: "ml-jupyter-notebook"
  appCatalog: "local"
  roles: 
    - 
      id: "controller"
      members: 1
      resources: 
        requests: 
          memory: "4Gi"
          cpu: "2"
          nvidia.com/gpu: "0"
        limits: 


## Summary

In this tutorial, you learned how **tenant users** can deploy, programmatically, cloud-native stateless, microservices applications, and non-cloud-native distributed stateful AI/analytics KubeDirector applications on Kubernetes clusters managed by HPE Container Platform. You also used the standard Kubernetes command line (kubectl) to interact with the Kubernetes cluster in the context of your tenant user account. 

* [Conclusion](3-Conclusion.ipynb)